In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn.utils import shuffle

# subset100 = pd.read_csv("../raw_data/track_meta_milestone3.csv", index_col="Unnamed: 0")
subset100 = pd.read_csv("../raw_data/track_meta_100subset_new.csv")
subset100 = shuffle(subset100)
# subset100 = subset100.reset_index()

In [2]:
subset100.shape

(3079, 28)

In [3]:
subset100['Playlistid'].nunique()

100

### Train-val-test split

In [4]:
# Train-val-test split (20%)
train, test = train_test_split(subset100, test_size=0.2, random_state=42, stratify = subset100['Playlistid'])
train, val = train_test_split(train, test_size=0.2, random_state=42, stratify = train['Playlistid'])

In [5]:
# Drop features here
features_drop = ["Playlistid","Playlist","Album", "Track", "Artist", "Trackid", "Artist_Name", "Track_Name", "Album_Name", "Artist_uri", "Track_uri", "Album_uri", "artist_genres", "explicit"]
train_cleaned, val_cleaned, test_cleaned = train.drop(features_drop, axis =1), val.drop(features_drop, axis=1), test.drop(features_drop, axis=1)

In [6]:
train = train.reset_index(drop=True)
train_cleaned = train_cleaned.reset_index(drop=True)

In [7]:
train_cleaned.head()

,Track_Duration,acousticness,artist_popularity,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,181573,0.000107,100,0.766,0.442,0.000061,1,0.1110,-8.558,1,0.3560,201.800,4,0.3900
1,310720,0.015200,88,0.587,0.698,0.000000,1,0.4420,-8.948,1,0.0943,133.954,4,0.0985
2,278573,0.433000,84,0.607,0.784,0.000000,7,0.6890,-6.293,1,0.2680,81.004,4,0.4820
3,229525,0.369000,86,0.689,0.481,0.000001,10,0.0649,-7.503,1,0.0815,80.025,4,0.2830
4,448533,0.131000,81,0.690,0.718,0.000729,4,0.6150,-4.743,0,0.1350,76.007,4,0.4010


### Content-based Collaborative Filtering

### Create a cosine-similarity matrix

In [8]:
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.preprocessing import MinMaxScaler

# Standardize the data
scaler = MinMaxScaler()
scaler.fit(train_cleaned)
train_scaled = scaler.transform(train_cleaned)
test_scaled = scaler.transform(test_cleaned)

In [9]:
train_cleaned.head()

,Track_Duration,acousticness,artist_popularity,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,181573,0.000107,100,0.766,0.442,0.000061,1,0.1110,-8.558,1,0.3560,201.800,4,0.3900
1,310720,0.015200,88,0.587,0.698,0.000000,1,0.4420,-8.948,1,0.0943,133.954,4,0.0985
2,278573,0.433000,84,0.607,0.784,0.000000,7,0.6890,-6.293,1,0.2680,81.004,4,0.4820
3,229525,0.369000,86,0.689,0.481,0.000001,10,0.0649,-7.503,1,0.0815,80.025,4,0.2830
4,448533,0.131000,81,0.690,0.718,0.000729,4,0.6150,-4.743,0,0.1350,76.007,4,0.4010


In [10]:
train_scaled[:5]

array([[2.29438009e-01, 1.03398791e-04, 1.00000000e+00, 7.42405832e-01,
        4.41181560e-01, 6.48305085e-05, 9.09090909e-02, 9.66702470e-02,
        7.91658637e-01, 1.00000000e+00, 4.24570337e-01, 9.47141244e-01,
        7.50000000e-01, 3.76119718e-01],
       [4.33547800e-01, 1.55518045e-02, 8.70967742e-01, 5.24908870e-01,
        7.01759922e-01, 0.00000000e+00, 9.09090909e-02, 4.52201933e-01,
        7.80922230e-01, 1.00000000e+00, 9.14067473e-02, 4.93816107e-01,
        7.50000000e-01, 6.89219096e-02],
       [3.82741219e-01, 4.43190041e-01, 8.27956989e-01, 5.49210207e-01,
        7.89297965e-01, 0.00000000e+00, 6.36363636e-01, 7.17508056e-01,
        8.54012388e-01, 1.00000000e+00, 3.12539784e-01, 1.40021248e-01,
        7.50000000e-01, 4.73074086e-01],
       [3.05223530e-01, 3.77682987e-01, 8.49462366e-01, 6.48845687e-01,
        4.80879045e-01, 1.09110169e-06, 9.09090909e-01, 4.71535983e-02,
        8.20701996e-01, 1.00000000e+00, 7.51113940e-02, 1.33479885e-01,
        7.500

In [11]:
train_scaled_cos_matrix = cosine_similarity(train_scaled)
# np.dot(np.array(trial_scaled[0,]), np.array(trial_scaled[5,]))/ (np.linalg.norm(trial_scaled[0,]) * np.linalg.norm(trial_scaled[5,]))


In [12]:
train_scaled_cos_matrix.shape

(1970, 1970)

In [13]:
train_scaled_cos_matrix[:5]

array([[1.        , 0.92959105, 0.83549703, ..., 0.85600866, 0.72873387,
        0.82150808],
       [0.92959105, 1.        , 0.91434055, ..., 0.89222469, 0.740573  ,
        0.81868149],
       [0.83549703, 0.91434055, 1.        , ..., 0.92754218, 0.79770255,
        0.84392153],
       [0.8356399 , 0.8676911 , 0.93026882, ..., 0.93188371, 0.82829977,
        0.82447315],
       [0.73853673, 0.82789796, 0.87374788, ..., 0.8030024 , 0.61454312,
        0.95046774]])

In [14]:
display(subset100.head())
subset100.shape

,Playlistid,Trackid,Artist_Name,Track_Name,Album_Name,Track_Duration,Artist_uri,Track_uri,Album_uri,acousticness,...,loudness,mode,speechiness,tempo,time_signature,valence,Playlist,Album,Track,Artist
1696,181874,3,Nickelback,Photograph,All The Right Reasons,258920,spotify:artist:6deZN1bslXzeGvOLaLMOIF,spotify:track:3hb2ScEVkGchcAQqrPLP0R,spotify:album:3eZd2XbhLyPcgbgcsLTZh3,0.000932,...,-3.756,1,0.0292,145.919,4,0.389,alternative,25,34,12
1802,190574,4,Creedence Clearwater Revival,Fortunate Son,Willy And The Poor Boys,140773,spotify:artist:3IYUhFvPQItj6xySrBmZkd,spotify:track:4BP3uh0hFLFRb5cjsgLqDh,spotify:album:31q47gQszFt0CddSyMksgO,0.201000,...,-7.516,1,0.0374,132.770,4,0.663,feel good,13,16,12
1069,92360,46,B.o.B,Magic (feat. Rivers Cuomo),B.o.B Presents: The Adventures of Bobby Ray,196133,spotify:artist:5ndkK3dpZLKtBklKjxNQwT,spotify:track:5uHYcK0nbEYgRaFTY5BqnP,spotify:album:7apLPYT8szV1IqTxyVSy5P,0.012700,...,-4.110,0,0.3430,82.439,4,0.787,Throwback,70,77,56
677,56681,0,Ginuwine,Pony,R&B: From Doo-Wop To Hip-Hop,251733,spotify:artist:7r8RF1tN2A4CiGEplkp1oP,spotify:track:3pfXxHoydFRfD7IBGJTQAN,spotify:album:3O2912EZv0FBJKZ0K5ruzK,0.001860,...,-9.359,0,0.0860,142.024,4,0.966,Marathon,27,28,22
358,11136,7,One Direction,Drag Me Down,Made In The A.M.,192120,spotify:artist:4AK6F7OLvEQ5QYCBNiQWHq,spotify:track:2K87XMYnUMqLcX3zvtAF4G,spotify:album:1gMxiQQSg5zeu4htBosASY,0.109000,...,-5.672,0,0.0369,138.113,4,0.595,yeah,16,16,15


(3079, 28)

In [53]:
def cos_similar_songs_playlist(cos_matrix, orig_df, target_playlist_id, cand_list_size):
    """
    Input:
    cos_matrix: cosine matrix of the tracks
    orig_df: original df with tracks as rows, but with playlistid and other features (e.g., train)
    target_playlist_id: id of the target playlist
    target_playlist_inx: index of playlist in the training set
    cand_list_size: candidate list of songs to recommend size (= test-set size * 15)
    
    Output:
    k_song_to_recommend: the most similar tracks per track
    """
    target_track_inx = np.where(train["Playlistid"] == target_playlist_id)[0] # index of tracks in training playlist of target playlist

    # remove rows in matrix with songs that live in the same playlist with the target song
    # candidate_cos_matrix = np.delete(cos_matrix, target_track_inx, 1) # remoe songs that co-exist in the same playlist
    candidate_cos_matrix = cos_matrix

    ## For each song in the playlist, find k similar songs
    cand_list = []
    # cand_list_size = k*15
    # taget_track_inx
    k = np.floor(cand_list_size/len(target_track_inx)) # round(cand_list_size/len(target_track_inx))
    k_rest = cand_list_size - k*len(target_track_inx)
    # e.g., for a candidate list size of 30, get 3 songs for each track first
    for inx, i in enumerate(target_track_inx):
        candidate_song_rec = candidate_cos_matrix[i, ] #ith row of matrix
        candidate_song_rec_inx = np.argsort(candidate_song_rec)
        unique_candidate_song_sorted = train['Track_uri'][candidate_song_rec_inx][::-1].drop_duplicates()
        tracks_in_target_playlist = train.loc[train["Playlistid"] == target_playlist_id, "Track_uri"]
        song_to_recommend = np.array(unique_candidate_song_sorted.loc[~unique_candidate_song_sorted.isin(tracks_in_target_playlist)])

        if (k_rest != 0 & inx <= k_rest): # 30-24 = 6; for the first 6 tracks recommend k + 1 songs
            k_song_to_recommend = song_to_recommend[:int(k+1)]
        else:
            k_song_to_recommend = song_to_recommend[:int(k)]
            
        if inx == 0:
            cand_list = k_song_to_recommend
        else:
            cand_list = np.append(cand_list, k_song_to_recommend)
    return list(cand_list) # turn np array into list

In [16]:
train.head()

,Playlistid,Trackid,Artist_Name,Track_Name,Album_Name,Track_Duration,Artist_uri,Track_uri,Album_uri,acousticness,...,loudness,mode,speechiness,tempo,time_signature,valence,Playlist,Album,Track,Artist
0,61388,67,Drake,The Motto,Take Care,181573,spotify:artist:3TVXtAsR1Inumwj472S9r4,spotify:track:4Kz4RdRCceaA9VgTqBhBfa,spotify:album:6X1x82kppWZmDzlXXK3y3q,0.000107,...,-8.558,1,0.3560,201.800,4,0.3900,OVO,39,95,18
1,51590,17,Kendrick Lamar,"Bitch, Don’t Kill My Vibe","good kid, m.A.A.d city",310720,spotify:artist:2YZyLoL8N0Wb9xBt1NhZWg,spotify:track:4Pwjz3DfvfQWV0rO2V8jyh,spotify:album:6PBZN8cbwkqm1ERj2BGXJ1,0.015200,...,-8.948,1,0.0943,133.954,4,0.0985,Rap,30,38,23
2,193450,47,J. Cole,Crooked Smile,Born Sinner,278573,spotify:artist:6l3HvQ5sa6mXTsMTB19rO5,spotify:track:6JG0qhINKVwiHxqN85j7RG,spotify:album:1NfrmcXk8xNennyxQ57JcW,0.433000,...,-6.293,1,0.2680,81.004,4,0.4820,Fall 2016,51,55,45
3,198885,15,Wiz Khalifa,"See You Again (feat. Charlie Puth) - From ""Fur...",Nine Track Mind,229525,spotify:artist:137W8MRPWKqSmrBGDBFSop,spotify:track:6IcrwGdIvRsrImOnoYfG9Z,spotify:album:5J4SS8wTmXdyIEVYjmHzpZ,0.369000,...,-7.503,1,0.0815,80.025,4,0.2830,my favorite music,22,27,18
4,73524,0,Justin Timberlake,What Goes Around.../...Comes Around (Interlude),FutureSex/LoveSounds,448533,spotify:artist:31TPClRtHm23RisEBtV3X7,spotify:track:4VZmWrPYFJP8T17AciiQJh,spotify:album:3EaBZNWtqfr7Ju5iTrctnB,0.131000,...,-4.743,0,0.1350,76.007,4,0.4010,Barry,6,8,4


In [19]:
pi = 61388
cos_matrix = train_scaled_cos_matrix
orig_df = train
target_playlist_id = pi
cand_list_size = nholdout(pi, val)*15

In [22]:
target_track_inx = np.where(train["Playlistid"] == target_playlist_id)[0]
target_track_inx

array([   0,   70,   95,  122,  127,  133,  135,  185,  290,  318,  380,
        440,  486,  488,  499,  524,  537,  542,  636,  652,  706,  733,
        739,  754,  791, 1065, 1073, 1087, 1097, 1139, 1156, 1227, 1241,
       1263, 1266, 1299, 1325, 1358, 1462, 1482, 1484, 1502, 1526, 1561,
       1575, 1579, 1616, 1622, 1623, 1664, 1672, 1673, 1745, 1754, 1763,
       1781, 1790, 1855, 1865, 1923, 1954])

In [23]:
target_track_inx = np.where(train["Playlistid"] == target_playlist_id)[0]
candidate_cos_matrix = cos_matrix

## For each song in the playlist, find k similar songs
cand_list = []
# cand_list_size = k*15
# taget_track_inx
k = np.floor(cand_list_size/len(target_track_inx)) # round(cand_list_size/len(target_track_inx))
k_rest = cand_list_size - k*len(target_track_inx)

In [25]:
k, k_rest

(3.0, 42.0)

In [26]:
list(enumerate(target_track_inx))

[(0, 0),
 (1, 70),
 (2, 95),
 (3, 122),
 (4, 127),
 (5, 133),
 (6, 135),
 (7, 185),
 (8, 290),
 (9, 318),
 (10, 380),
 (11, 440),
 (12, 486),
 (13, 488),
 (14, 499),
 (15, 524),
 (16, 537),
 (17, 542),
 (18, 636),
 (19, 652),
 (20, 706),
 (21, 733),
 (22, 739),
 (23, 754),
 (24, 791),
 (25, 1065),
 (26, 1073),
 (27, 1087),
 (28, 1097),
 (29, 1139),
 (30, 1156),
 (31, 1227),
 (32, 1241),
 (33, 1263),
 (34, 1266),
 (35, 1299),
 (36, 1325),
 (37, 1358),
 (38, 1462),
 (39, 1482),
 (40, 1484),
 (41, 1502),
 (42, 1526),
 (43, 1561),
 (44, 1575),
 (45, 1579),
 (46, 1616),
 (47, 1622),
 (48, 1623),
 (49, 1664),
 (50, 1672),
 (51, 1673),
 (52, 1745),
 (53, 1754),
 (54, 1763),
 (55, 1781),
 (56, 1790),
 (57, 1855),
 (58, 1865),
 (59, 1923),
 (60, 1954)]

In [27]:
cos_matrix.shape

(1970, 1970)

In [28]:
train.shape

(1970, 28)

In [29]:
inx = 0
i = 0 #track inx in cos matrix
candidate_song_rec = candidate_cos_matrix[i, ] #ith row of matrix
candidate_song_rec_inx = np.argsort(candidate_song_rec) # ascending order of similiarity (last item is most similar)
unique_candidate_song_sorted = train['Track_uri'][candidate_song_rec_inx][::-1].drop_duplicates()
tracks_in_target_playlist = train.loc[train["Playlistid"] == target_playlist_id, "Track_uri"]
song_to_recommend = np.array(unique_candidate_song_sorted.loc[~unique_candidate_song_sorted.isin(tracks_in_target_playlist)])

In [35]:
display(candidate_song_rec)
candidate_song_rec_inx

array([1.        , 0.92959105, 0.83549703, ..., 0.85600866, 0.72873387,
       0.82150808])

array([ 208,  617, 1699, ...,  486,  589,    0])

In [36]:
song_to_recommend

array(['spotify:track:0VgkVdmE4gld66l8iyGjgx',
       'spotify:track:6HZILIRieu8S0iqY8kIKhj',
       'spotify:track:14RRHdmfjGPhuWBhkl4Ne4', ...,
       'spotify:track:6tCUa4YpFXV4sTUmdg38Vb',
       'spotify:track:3weNRklVDqb4Rr5MhKBR3D',
       'spotify:track:3fXJILbvVqkG9ycYYdD3H1'], dtype=object)

In [45]:
unique_playlistid = train['Playlistid'].drop_duplicates()

In [46]:
unique_playlistid

0        61388
1        51590
2       193450
3       198885
4        73524
5       223534
6       231023
7         2259
8       175237
9       100221
10      256979
12      206080
13      208692
14      151474
15        2535
17       67941
18      114695
20      210308
21       68119
22      221044
23       38828
25       59745
27      117841
28      107065
29      190503
30      211406
31      190574
32      118724
33      229646
35       20043
         ...  
150     196206
154     242823
156     216556
161     123429
162     219212
163     159326
186     120613
187      81542
191     248553
213     252197
224      11119
226     107941
250      10254
252      97870
255      46047
257      77069
300     268318
308     208779
327      90187
335      48330
341     196255
345     182533
379     232421
381     238096
410      62657
434      41347
625     155081
664     161016
946     217189
1130     79469
Name: Playlistid, Length: 100, dtype: int64

## Making Predictions

In [18]:
def nholdout(playlist_id, df):
    '''Pass in a playlist id to get number of songs held out in val/test set'''
    
    return len(df[df.Playlistid == playlist_id].Track_uri)

In [ ]:
train.loc[train["Playlistid"] == target_playlistid, "Track_uri"]

In [54]:
### Prediction Example
pi = 61388 # target playlist index
kpreds = cos_similar_songs_playlist(train_scaled_cos_matrix, train, pi, nholdout(pi, val)*15)

In [55]:
type(kpreds)

list

In [56]:
kpreds

['spotify:track:0VgkVdmE4gld66l8iyGjgx',
 'spotify:track:6HZILIRieu8S0iqY8kIKhj',
 'spotify:track:14RRHdmfjGPhuWBhkl4Ne4',
 'spotify:track:7zxRMhXxJMQCeDDg0rKAVo',
 'spotify:track:0wdKiSBUT7aZkXUIdJWcwC',
 'spotify:track:1D066zixBwqFYqBhKgdPzp',
 'spotify:track:4ejtrks9KfRe7keI7FRIPc',
 'spotify:track:3AYcyxEACnmE6d96RPubID',
 'spotify:track:2IGr3q6ljcpLvHDFLtI0yW',
 'spotify:track:6bxUnsSGZCmoHHU5auwtps',
 'spotify:track:6rbeWjEavBHvX2kr6lSogS',
 'spotify:track:27GmP9AWRs744SzKcpJsTZ',
 'spotify:track:4kNvYhyl8R6m1vykVkcuBu',
 'spotify:track:6fAyRZ0GHuuvSOEIOwi58N',
 'spotify:track:0Fv5N0cHBsl4bzCbollCAS',
 'spotify:track:1nX9KhK3Fff27SnrIor2Yb',
 'spotify:track:3B7udSGy2PfgoCniMSb523',
 'spotify:track:35WhawODuOs0kaHxwmPA9D',
 'spotify:track:047fCsbO4NdmwCBn8pcUXl',
 'spotify:track:2fbXJ0VpxhW7j0qcg1DnoZ',
 'spotify:track:2Nfd2ITp4WkI1aoZlYpZwg',
 'spotify:track:0VgkVdmE4gld66l8iyGjgx',
 'spotify:track:6UjfByV1lDLW0SOVQA4NAi',
 'spotify:track:0WCbhE2evMrIwRM0DlMy9k',
 'spotify:track:

In [57]:
val_set = val[val.Playlistid == pi]
val_set = val_set['Track_uri'] # ground truth

## Metrics

In [58]:
def r_precision(prediction, val_set):
# prediction should be a list of predictions
# val_set should be pandas Series of ground truths
    score = np.sum(val_set.isin(prediction))/val_set.shape[0]
    return score

In [60]:
### Example Usage
r_precision(kpreds, val_set)

0.26666666666666666

In [61]:
### NDCG Code Source: https://gist.github.com/bwhite/3726239
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, k, method=0):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [62]:
### Example Usage
# Generate binary relevance array
r = np.zeros(len(kpreds))
for i, p in enumerate(kpreds):
    if p in val_set:
        r[i] = 1

ndcg_at_k(r, len(r))

0.0

## Baseline Model Performance

In [63]:
unique_playlistid = train['Playlistid'].drop_duplicates()
unique_playlistid

0        61388
1        51590
2       193450
3       198885
4        73524
5       223534
6       231023
7         2259
8       175237
9       100221
10      256979
12      206080
13      208692
14      151474
15        2535
17       67941
18      114695
20      210308
21       68119
22      221044
23       38828
25       59745
27      117841
28      107065
29      190503
30      211406
31      190574
32      118724
33      229646
35       20043
         ...  
150     196206
154     242823
156     216556
161     123429
162     219212
163     159326
186     120613
187      81542
191     248553
213     252197
224      11119
226     107941
250      10254
252      97870
255      46047
257      77069
300     268318
308     208779
327      90187
335      48330
341     196255
345     182533
379     232421
381     238096
410      62657
434      41347
625     155081
664     161016
946     217189
1130     79469
Name: Playlistid, Length: 100, dtype: int64

In [67]:
rps = []
ndcgs = []
for pid in unique_playlistid: # loop through each playlist
    print(pid)
    ps = cos_similar_songs_playlist(train_scaled_cos_matrix, train, pi, nholdout(pi, val)*15)# predictions
    vs = val[val.Playlistid == pid].Track_uri # ground truth
    
    print(r_precision(ps, vs))
    rps.append(r_precision(ps, vs)) # append individual r-precision score
    
    # NDCG
    r = np.zeros(len(ps))
    for i, p in enumerate(ps):
        if np.any(vs.isin([p])):
            r[i] = 1
    ndcgs.append(ndcg_at_k(r, len(r)))
    

61388
0.26666666666666666
51590
0.0
193450
0.1111111111111111
198885
0.0
73524
0.0
223534
0.0
231023
0.16666666666666666
2259
0.0
175237
0.0
100221
0.0
256979
0.2222222222222222
206080
0.0
208692
0.2727272727272727
151474
0.0
2535
0.0
67941
0.0
114695
0.0
210308
0.0
68119
0.0
221044
0.14285714285714285
38828
0.1
59745
0.0
117841
0.0
107065
0.0
190503
0.0
211406
0.0
190574
0.3333333333333333
118724
0.0
229646
0.0
20043
0.2727272727272727
241349
0.0
622
0.0
430
0.0
92360
0.07692307692307693
33568
0.0
4575
0.0
22204
0.16666666666666666
56681
0.0
241546
0.0
168982
0.3333333333333333
164819
0.0
155598
0.0
11136
0.0
181874
0.2
116737
0.0
118342
0.0
195247
0.0
120354
0.16666666666666666
6238
0.0
151748
0.0
89825
0.0
174875
0.0
167436
0.0
249966
0.0
266156
0.0
186672
0.0
271274
0.0
201186
0.0
72703
0.0
89955
0.0
237553
0.1111111111111111
1990
0.0
193016
0.0
58121
0.0
153929
0.23076923076923078
230183
0.2
204668
0.0
37634
0.25
42049
0.0
194212
0.0
196206
0.0
242823
0.25
216556
0.0
123429
0.0
21

In [68]:
avg_rp = np.mean(rps)
avg_ndcg = np.mean(ndcgs)
print('Avg. R-Precision: ', avg_rp)
print('Avg. NDCG: ', avg_ndcg)
print('Total Sum: ', np.mean([avg_rp, avg_ndcg]))

Avg. R-Precision:  0.06573781773781774
Avg. NDCG:  0.04372503207052835
Total Sum:  0.05473142490417304
